In [1]:
import sys
sys.path.append(r'..//')
from ioMicro import *

In [2]:
def new_segmentation_subtr(fl =r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\H1_MER_set1\Conv_zscan__030.zarr',
                     psf_file = '\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy',
                           subtr_bk=True,
                     p1=-500,p99=1500,mean_dapi = None,sdapi = 100,
                    save_folder = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\MERFISH_Analysis',redo=False,plt_val=False,st = 5):

    segm_folder = save_folder+os.sep+'Segmentation'
    if not os.path.exists(segm_folder): os.makedirs(segm_folder)
    fl_dapi = fl
    save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
    if redo or (not os.path.exists(save_fl)):
    
        im = read_im(fl)
        im_dapi = np.array(im[-1],dtype=np.float32)
        imd = im_dapi
        if subtr_bk:
            im_bk = np.array(im[-2],dtype=np.float32)
            ifr = len(im_dapi)//2
            fr = im_bk[ifr]/im_dapi[ifr]
            fr_ = np.max(fr)
            im_dapi = im_dapi-im_bk/fr_
        
        if psf_file is not None:
            psf = np.load(psf_file)
            imd = full_deconv(im_dapi,s_=500,pad=100,psf=psf,parameters={'method': 'wiener', 'beta': 0.01},gpu=True,force=False)
        
        im_dapi_ = norm_slice(imd,s=sdapi)


        img = np.array(np.clip((im_dapi_[::3,::4,::4]-p1)/(p99-p1),0,1),dtype=np.float32)
        imd_ = imd[::3,::4,::4]

        from cellpose import models, io,utils
        from scipy import ndimage
        model = models.Cellpose(gpu=True, model_type='cyto')
        masks, flows, styles, diams = model.eval(img,z_axis=0, diameter=20, channels=[0,0],
                                                 flow_threshold=-10,cellprob_threshold=-10,normalize=False,
                                                 do_3D=False,stitch_threshold=0.,
                                                 progress=True)

        for ifr in range(len(masks)):
            means = nd.mean(np.clip(imd_[ifr],0,np.max(imd_[ifr])),masks[ifr],np.unique(masks[ifr]))
            
            if mean_dapi is None: mean_dapi = (st*means[0]+np.median(means[1:]))/(st+1)
            bad_cells = np.where(means<mean_dapi)[0]
            masks[ifr] = replace_mat(masks[ifr],bad_cells,0)
        if plt_val:
            import napari
            v = napari.view_image(img)
            v.add_labels(masks)
        shape = np.array(im[-1].shape)
        np.savez_compressed(save_fl,segm = masks,shape = shape)
    return save_fl

In [3]:
fl =r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\H0__set1\Conv_zscan__030.zarr'
im = read_im(fl)
import napari
napari.view_image(im);

In [4]:
#save_folder =r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis'
save_folder = master_data_folder = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023'
if True:
    fl = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\H0__set1\Conv_zscan__030.zarr'
    new_segmentation_subtr(fl =fl,subtr_bk=True,
                             psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=5000,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=True,plt_val=True,st=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:08<00:00,  4.50it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 25.3M/25.3M [00:01<00:00, 13.3MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 5.23k/5.23k [00:00<?, ?B/s]


In [ ]:
r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\H0__set1\Conv_zscan__030.zarr'

In [5]:
fls_all = glob.glob(r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\H0__set1\*.zarr')
save_folder =r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023'
for fl in tqdm(fls_all[::-1]):
    try:
        new_segmentation_subtr(fl =fl,subtr_bk=True,
                             psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=5000,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=False,plt_val=False,st=1)
    except:
        print("Failed",fl)

100%|██████████████████████████████████████████████████████████████████████████████| 336/336 [2:26:44<00:00, 26.20s/it]
